<a href="https://colab.research.google.com/github/Sc0rpie/VU_DeepLearning/blob/main/Lab1_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pirmas laboratorinis darbas

Olen Račkauskas, 3 grupė, 2 pogrupis, 2213787 (VGG16)

Klasifikuojamos klasės: Orange, Banana, Strawberry

In [ ]:
!pip install openimages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.9 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

Atsiunčiamos nuotraukos (300 Orange ir Banana, 400 Strawberry = 1000)

In [ ]:
data_dir = "data"
number_for_samples = 300
classes_to_download = ["Orange", "Banana"]
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

print("Download commencing...")
from openimages.download import download_dataset
download_dataset(data_dir, classes_to_download, limit=number_for_samples)
download_dataset(data_dir, ["Strawberry"], limit=400)
print("Download finished.")

Download commencing...


100%|██████████| 400/400 [00:25<00:00, 15.89it/s]


Download finished.


Sukomponuojam transformacijas bei panaudojam jau aprašyta DataSet: ImageFolder

Prie to pačio patikrinam ar turim 1000 nuotraukų

In [ ]:
all_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
dataset = datasets.ImageFolder(root=data_dir, transform=all_transforms)
print(f"Dataset size: {len(dataset)}")

Dataset size: 1000


Paruošiamas DataLoader bei peržiūrimas klasių mapping'as

In [ ]:
loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

classes = sorted(os.listdir('data'))
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
print("Class to index mapping:", class_to_idx)

Class to index mapping: {'banana': 0, 'orange': 1, 'strawberry': 2}


Paruošiamas VGG16 modelis naudojimui

In [ ]:
model = models.vgg16(pretrained=True)
model.to("cuda")
model.eval();

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 77.5MB/s]


Aprašomos klasių ID iš imagenet 1000 klasių sąrašo

In [ ]:
dataset_to_imagenet = {
    0: 954,   # "Banana"
    1: 950,   # "Orange"
    2: 949    # "Strawberry"
}

In [ ]:
Aprašomos funkcijos skaičiavimams

In [ ]:
def compute_metrics(ground_truth, predictions, threshold=0.5):
    pred_bin = (predictions > threshold).astype(np.float64)
    TP = np.sum((ground_truth == 1) & (pred_bin == 1))
    TN = np.sum((ground_truth == 0) & (pred_bin == 0))
    FP = np.sum((ground_truth == 0) & (pred_bin == 1))
    FN = np.sum((ground_truth == 1) & (pred_bin == 0))
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    return {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'f1': f1}

def get_class_predictions(prob_array, labels, imagenet_class_index, dataset_label):
    gt = (labels == dataset_label).astype(np.float64)
    pred = prob_array[:, imagenet_class_index]
    return gt, pred

Paleidžiamas modelis, įrašomos visos tikimybės į masyvą

In [ ]:
all_probabilities = []
all_labels = []

for images, labels in loader:
    images, labels = images.cuda(), labels.cuda()
    outputs = model(images)
    probabilities = torch.softmax(outputs, dim=1).detach().cpu().numpy()
    all_probabilities.append(probabilities)
    all_labels.append(labels.cpu().numpy())

all_probabilities = np.concatenate(all_probabilities, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

Gauti rezultatai naudojami skaičiavimams ir rezultato atvaizdavimui

In [ ]:
metrics_results = {}
for dataset_label, imagenet_idx in dataset_to_imagenet.items():
    gt, pred = get_class_predictions(all_probabilities, all_labels, imagenet_idx, dataset_label)
    metrics = compute_metrics(gt, pred, threshold=0.4)
    class_name = list(class_to_idx.keys())[list(class_to_idx.values()).index(dataset_label)]
    metrics_results[class_name] = metrics

for class_name, metrics in metrics_results.items():
    print(f"{class_name} metrics: {metrics}")

banana metrics: {'accuracy': 0.879, 'recall': 0.5966666666666667, 'precision': 1.0, 'f1': 0.7473903966597077}
orange metrics: {'accuracy': 0.81, 'recall': 0.37333333333333335, 'precision': 0.9824561403508771, 'f1': 0.5410628019323672}
strawberry metrics: {'accuracy': 0.769, 'recall': 0.4225, 'precision': 1.0, 'f1': 0.5940246045694201}
